### Exploratory Data Analysis
This notebook provides an exploratory analysis of the cleaned Online Retail Transaction dataset. The goal is to uncover key patterns in customer behaviour, product performance, and sales trends to support analysis and dashboard development.

In [2]:
# Import necessary libraries
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

Now let us import our cleaned dataset for analysis:

In [3]:
# Import the cleaned dataset
df = pd.read_csv('../data/clean_data/online_retail_cleaned.csv')
# Display the first few rows of the dataset
df.head()

C:\Users\hidde\AppData\Local\Temp\ipykernel_8344\993137673.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/clean_data/online_retail_cleaned.csv')


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice,Year,Month,DayOfWeek,Hour,Date
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30,2010,12,Wednesday,8,2010-12-01
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,2010,12,Wednesday,8,2010-12-01
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00,2010,12,Wednesday,8,2010-12-01
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,2010,12,Wednesday,8,2010-12-01
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,2010,12,Wednesday,8,2010-12-01


The first stage of our analysis is to look at total revenue over time. 
We can do this by plotting an initial line chart visualisation. 

In [4]:
daily_sales = df.groupby('Date')['TotalPrice'].sum().reset_index()

fig = px.line(daily_sales, x='Date', y='TotalPrice', title='Daily Sales Revenue')
fig.show()

This first plot shows our Daily Sales Revenue over time. As we can see, it has a very 'spiky' appearance. In order to help combat this, it can be useful to add a rolling average line to the plot to help us smooth out the plot so that we can see changes over time more effectively. 

In [5]:
daily_sales['Rolling7Day'] = daily_sales['TotalPrice'].rolling(window=7).mean()

import plotly.express as px
fig = px.line(daily_sales, x='Date', y=['TotalPrice', 'Rolling7Day'],
              labels={'value': 'Revenue', 'variable': 'Line'},
              title='Daily Sales with 7-Day Rolling Average')
fig.show()

We observe that there are revenue spikes in December 2010, and a significant spike in December 2011, most likely indicating Christmas/Holidays sales.
We also see a spike around mid Jan 2011, possibly indicating post-holidays spends or returns. 
September 2011 and November 2011 also see revenue spikes. This could indicate early holiday period spending, back to school spending, spending for 'Thanksgiving' in November, or 'Cyber Monday' sales.

Next we can check to see which countries contribute the most to revenues. 
Let us make a bar plot of revenue by country.

In [6]:
# Group by Country and sum TotalPrice
country_sales = df.groupby('Country')['TotalPrice'].sum().reset_index()

# Sort by revenue
country_sales = country_sales.sort_values(by='TotalPrice', ascending=False)

# Plot
fig = px.bar(country_sales, x='Country', y='TotalPrice',
             title='Total Revenue by Country',
             labels={'TotalPrice': 'Total Revenue'},
             height=600)
fig.update_layout(xaxis_tickangle=45)
fig.show()

We can see from our bar plot the UK is our Primary Market, dominating revenues with sales ~£9M. This could mean that the operations are UK-centric. 
This highlights the retailer's dependence on it's domestic market, and suggests that international markets are underdeveloped or under-targetted.
It might be useful in this instance to evaluate international sales by excluding the UK.

In [7]:
# Exclude the UK for better visual balance
non_uk_sales = country_sales[country_sales['Country'] != 'United Kingdom']

fig = px.bar(non_uk_sales.sort_values(by='TotalPrice', ascending=False),
             x='Country', y='TotalPrice',
             title='Revenue by Country (Excluding UK)',
             labels={'TotalPrice': 'Total Revenue'},
             height=600)
fig.update_layout(xaxis_tickangle=45)
fig.show()

Unsurprisingly, we can see from this bar plot, that our secondary markets are situated within the European Union. 
At this point, it could be helpful to our analysis to create a dictionary to group our countries into regions. 

In [8]:
# Define region mapping
# This dictionary maps countries to their respective regions for better analysis

region_map = {
    'United Kingdom': 'UK',
    'France': 'Europe',
    'Germany': 'Europe',
    'Netherlands': 'Europe',
    'Belgium': 'Europe',
    'Switzerland': 'Europe',
    'Spain': 'Europe',
    'Portugal': 'Europe',
    'Italy': 'Europe',
    'Norway': 'Europe',
    'Austria': 'Europe',
    'Denmark': 'Europe',
    'Sweden': 'Europe',
    'Finland': 'Europe',
    'Ireland': 'Europe',
    'Greece': 'Europe',
    'Cyprus': 'Europe',
    'Channel Islands': 'Europe',

    'Australia': 'Oceania',
    'New Zealand': 'Oceania',

    'USA': 'North America',
    'Canada': 'North America',

    'Japan': 'Asia',
    'Hong Kong': 'Asia',
    'Singapore': 'Asia',
    'Israel': 'Middle East',
    'United Arab Emirates': 'Middle East',

    'Unspecified': 'Other',
    'EIRE': 'Europe'
}

Let us now create a column for 'Region' in our dataset:

In [9]:
# Create a Region column
df['Region'] = df['Country'].map(region_map).fillna('Other')

Now we can aggregate our revenues by region, and display our findings in a Bar Plot.

In [10]:
# Aggregate revenue by region
region_sales = df.groupby('Region')['TotalPrice'].sum().reset_index()

# Bar chart of regional revenue
fig = px.bar(region_sales.sort_values(by='TotalPrice', ascending=False),
             x='Region', y='TotalPrice',
             title='Revenue by Region',
             labels={'TotalPrice': 'Total Revenue'})
fig.show()

We can see here that our revenues by region are:

|__Region__|__Total Revenue__|
|----------|-----------------|
|UK|~£90M|
|Europe|~£1.4M|
|Oceania|~£138K|
|Asia|~£74K|
|Other|~£28K|
|Middle East|~£10K|
|North America|~£7K|

Let us also generate a Pie Chart for a more attractive visualisation of Revenue Share by Region.

In [11]:
fig = px.pie(region_sales, names='Region', values='TotalPrice', title='Revenue Share by Region')
fig.show()

Next we should look at which products are our top sellers by revenue, which products are the most frequently purchased (by quantity), are there products which sell well, but generate little revenue?


In [12]:
top_revenue_products = (
    df.groupby('Description')['TotalPrice']
    .sum()
    .sort_values(ascending=False)
    .head(10)
    .reset_index()
)

In [13]:
fig = px.bar(top_revenue_products, x='TotalPrice', y='Description',
             orientation='h',
             title='Top 10 Products by Revenue',
             labels={'TotalPrice': 'Revenue', 'Description': 'Product'})
fig.update_layout(yaxis={'categoryorder': 'total ascending'})
fig.show()

Here we can see that 'DOTCOM POSTAGE' is our top 'Product'. We can assume here that this 'product' is not actually a product, moreover a Postage and Packing charge.
We can also see 'POSTAGE' and 'Manual'. These are not relevant for our Product Level Analysis, so we should exclude them for our purposes.

In [14]:
non_products = ['DOTCOM POSTAGE', 'POSTAGE', 'Manual']

df_products = df[~df['Description'].isin(non_products)]

In [15]:
top_revenue_products = (
    df_products
    .groupby('Description')['TotalPrice']
    .sum()
    .sort_values(ascending=False)
    .head(10)
    .reset_index()
)

In [16]:
fig = px.bar(top_revenue_products, x='TotalPrice', y='Description',
             orientation='h',
             title='Top 10 Products by Revenue',
             labels={'TotalPrice': 'Revenue', 'Description': 'Product'})
fig.update_layout(yaxis={'categoryorder': 'total ascending'})
fig.show()

It is clear from this analysis that 'Homewares and Decorations' are our most popular type of items driving revenues across the business. 
Let's now check out what our lowest selling products are.

In [17]:
low_revenue_products = (
    df_products.groupby('Description')['TotalPrice']
    .sum()
    .sort_values()
    .head(10)
    .reset_index()
)

import plotly.express as px
fig = px.bar(low_revenue_products, x='TotalPrice', y='Description',
             orientation='h',
             title='Bottom 10 Products by Revenue',
             labels={'TotalPrice': 'Revenue', 'Description': 'Product'})
fig.show()

Now let us look at the products which have sold in the lowest quantities.

In [18]:
low_quantity_products = (
    df_products.groupby('Description')['Quantity']
    .sum()
    .sort_values()
    .head(10)
    .reset_index()
)

fig = px.bar(low_quantity_products, x='Quantity', y='Description',
             orientation='h',
             title='Bottom 10 Products by Quantity Sold',
             labels={'Quantity': 'Units Sold', 'Description': 'Product'})
fig.show()

So we can see from our lowest quantity of products purchased is 1. All of our bottom ten items have only sold a single unit throughout the whole year (2010-2011) that is covered by our dataset. These are clearly the products that perhaps our company should consider dropping from the inventory, or consider addressing pricing and marketing stategy in order to boost sales growth and revenue. 

Next we would like to look into products that have a low volume of sales and a high price. This could help to identify products that are unpopular, or to identify products that have premium pricing, are seen as niche, or overpriced. These products may represent luxury or specialised items, or products that take up inventory space without a strong Return On Investment (ROI).

In [21]:
# Create a summary DataFrame for products
# This DataFrame will aggregate total quantity, total revenue, and average unit price for each product
product_summary = (
    df_products.groupby('Description')
    .agg(TotalQuantity=('Quantity', 'sum'),
         TotalRevenue=('TotalPrice', 'sum'),
         AvgUnitPrice=('UnitPrice', 'mean'))
    .reset_index()
)

In [22]:
low_volume_high_price = product_summary[
    (product_summary['TotalQuantity'] < 20) &
    (product_summary['AvgUnitPrice'] > 10)
].sort_values(by='AvgUnitPrice', ascending=False)

low_volume_high_price.head(10)

,Description,TotalQuantity,TotalRevenue,AvgUnitPrice
263,Adjust bad debt,1,11062.06,11062.060000
155,AMAZON FEE,2,13761.09,6880.545000
2852,REGENCY MIRROR WITH SHUTTERS,10,1530.00,156.428571
1091,Dotcomgiftshop Gift Voucher Â£100.00,1,83.33,83.330000
3739,VINTAGE POST OFFICE CABINET,11,607.65,66.360000
3006,SCHOOL DESK AND CHAIR,19,1199.75,63.915385
967,DECORATIVE HANGING SHELVING UNIT,6,359.70,59.950000
476,BLUE PAINTED KASHMIRI CHAIR,3,149.85,49.950000
2588,PINK PAINTED KASHMIRI CHAIR,4,171.80,42.950000
1095,Dotcomgiftshop Gift Voucher Â£50.00,4,167.56,41.890000


Again, we will need to filter out non-physical products or accounting/financial entries in order to streamline our analysis and keep things at the product level.

In [ ]:
# Filter out non-products
non_products = [
    "Adjust bad debt",
    "Amazon Fee",
    "AMAZON FEE",
    "Dotcomgiftshop Gift Voucher A£100.00",
    "Dotcomgiftshop Gift Voucher A£50.00"
    "Dotcomgiftshop Gift Voucher Â£40.00",
    "Dotcomgiftshop Gift Voucher Â£50.00",
    "Dotcomgiftshop Gift Voucher Â£30.00",
    "Dotcomgiftshop Gift Voucher Â£20.00",
    "Dotcomgiftshop Gift Voucher Â£10.00",
    "Dotcomgiftshop Gift Voucher Â£100.00"
]

df_products = df_products[~df_products['Description'].isin(non_products)]

# Recalculate product summary
product_summary = (
    df_products.groupby('Description')
    .agg(TotalQuantity=('Quantity', 'sum'),
         TotalRevenue=('TotalPrice', 'sum'),
         AvgUnitPrice=('UnitPrice', 'mean'))
    .reset_index()
)

# Filter low volume, high price products
low_volume_high_price = product_summary[
    (product_summary['TotalQuantity'] < 20) &
    (product_summary['AvgUnitPrice'] > 10)
].sort_values(by='AvgUnitPrice', ascending=False)

# Display top 10
low_volume_high_price.head(10)

,Description,TotalQuantity,TotalRevenue,AvgUnitPrice
2845,REGENCY MIRROR WITH SHUTTERS,10,1530.00,156.428571
3732,VINTAGE POST OFFICE CABINET,11,607.65,66.360000
2999,SCHOOL DESK AND CHAIR,19,1199.75,63.915385
965,DECORATIVE HANGING SHELVING UNIT,6,359.70,59.950000
474,BLUE PAINTED KASHMIRI CHAIR,3,149.85,49.950000
2581,PINK PAINTED KASHMIRI CHAIR,4,171.80,42.950000
945,DANISH ROSE BEDSIDE CABINET,13,519.35,39.950000
3325,SILVER ROCCOCO CHANDELIER,14,504.32,39.396667
1392,FUSCHIA RETRO BAR STOOL,14,523.30,39.150000
681,CHALKBOARD KITCHEN ORGANISER,11,366.73,37.610000


To identify potentially underperforming or niche products, we analysed items that were sold in low quantities (fewer than 20 units) but had a high average unit price (greater than £10). These products may represent premium items, overpriced inventory, or non-essential goods with limited appeal.

After filtering out non-merchandise descriptors (such as shipping fees and gift vouchers), we identified several products with very high price points but low sales volumes. Examples include items like 'Regency Mirror w/ Shutters', 'Vintage Post Office Cabinet, and 'Blue Painted Kashmiri Chair', each commanding higher unit prices but selling only a small number of units across the entire dataset.

This insight may indicate:

- Niche appeal or exclusivity (deliberate low sales at high price)
- Pricing mismatch (potential for adjustment)
- Inventory cost inefficiency (depending on stockholding policies)

Recommendation:
These products may warrant a pricing review or marketing reevaluation, particularly if they occupy shelf space or incur warehousing costs. Alternatively, they may remain intentionally limited and should be assessed in the context of brand strategy.

### Customer Level Analysis

In this section, we are interested in examining questions such as:
- How many unique customers are in the dataset?
- How many are repeat buyers?
- What is the average number of orders per customer?
- Who are the most frequent buyers?

In [ ]:
# Count unique customers
unique_customers = df['CustomerID'].nunique()
print(f"Unique customers: {unique_customers}")

Unique customers: 4338


In [35]:
# Orders per customer (frequency of purchases)
customer_orders = df.groupby('CustomerID')['InvoiceNo'].nunique().reset_index(name='NumOrders')

# Summary stats
customer_orders.describe()

,CustomerID,NumOrders
count,4338.000000,4338.000000
mean,15300.408022,4.601660
std,1721.808492,22.989013
min,12346.000000,1.000000
25%,13813.250000,1.000000
50%,15299.500000,2.000000
75%,16778.750000,5.000000
max,18287.000000,1431.000000


Here we can see that there are 4338 unique customers in the dataset. Our average number of orders per customer is 4.6. 
We notice though that our minimum order is 1, and our max is 1431. This is quite a dramatic spread, so let us drill down a little more to find out who are our most loyal customers.

In [37]:
top_loyal_customers['CustomerID'] = top_loyal_customers['CustomerID'].astype(str)

fig = px.bar(top_loyal_customers, x='CustomerID', y='NumOrders',
             title='Top 10 Customers by Number of Orders',
             labels={'CustomerID': 'Customer ID', 'NumOrders': 'Number of Orders'})
fig.show()

We can now see that our most loyal customer is CustomerID: 15287 with 1431 orders.

In [38]:
# Add a loyalty category
customer_orders['LoyaltyCategory'] = customer_orders['NumOrders'].apply(
    lambda x: 'One-time' if x == 1 else 'Returning'
)

customer_orders['LoyaltyCategory'].value_counts()

LoyaltyCategory
Returning    2845
One-time     1493
Name: count, dtype: int64

Analysis of the customer base reveals that approximately two-thirds of all customers are repeat buyers, having placed more than one order during the observed period. 
This indicates a relatively strong level of customer retention, which may reflect positively on product satisfaction, brand engagement, or the effectiveness of marketing and follow-up strategies.

Conversely, around one-third of customers are one-time buyers, suggesting potential opportunities for post-purchase engagement, loyalty programs, or targeted reactivation campaigns.

This breakdown is visualised in the pie chart below, which categorises customers into ‘One-time’ and ‘Returning’ groups based on their order frequency.

In [39]:
fig = px.pie(customer_orders, names='LoyaltyCategory', title='Customer Loyalty Breakdown')
fig.show()

### Customer Level Analysis Summary
- How many unique customers are in the dataset?
Our analysis shows that for the examined period, we had 4338 unique customers.
- How many are repeat buyers?
Our analysis shows that ~65% or around two thirds of our customers are repeat buyers.
- What is the average number of orders per customer?
Our analysis shows an average of 4.6 orders per customer, with our most loyal customer placing 1431 orders over the examined period.
- Who are the most frequent buyers?
Our analysis shows that our most loyal customer is ID:15287

### Sales Heatmap 
As an additional insight to our analysis, it could be helpful to our business to examine peak shopping windows.
The heatmap below visualises total revenue by day of the week and hour of the day. This helps to identify when customers are most likely to make purchases. 

In [40]:
df['Hour'] = pd.to_datetime(df['InvoiceDate']).dt.hour
df['DayOfWeek'] = pd.to_datetime(df['InvoiceDate']).dt.day_name()

sales_by_time = df.groupby(['DayOfWeek', 'Hour'])['TotalPrice'].sum().reset_index()

import plotly.express as px
fig = px.density_heatmap(sales_by_time, x='Hour', y='DayOfWeek', z='TotalPrice',
                         title='Sales Heatmap by Day and Hour',
                         category_orders={'DayOfWeek': ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']})
fig.show()

### Key Observations
- Peak activity occurs on weekdays during mid-morning to early afternoon. Approximately 10am-2pm.
- Tuesdays, Wednesdays and Thursdays show strong performance, especially mid-morning. 
- Weekends, especially Saturdays and early mornings show lower activity, suggesting less engagement during non-working hours. 

These findings suggest that customer engagement is strongest during weekday business hours, particularly midweek. Promotional campaigns and email marketing may be most effective when scheduled during these peak windows.

### Key Takeaways from Exploratory Data Analysis
#### Customer Behaviour
- Approximately two-thirds of customers are repeat buyers, indicating a strong base of loyal customers.
- The top ten most loyal customers placed between 63-209 orders each over the analysis period, with the most loyal customer placing 1431 orders over the same period. 
- Customer loyalty is heavily concentrated in the UK, but international segments (especially Europe) may offer growth opportunities. 

#### Product Insights
- The top selling products by revenue include core merchandise, such as the Regency 3-tier Cakestand and the Papercraft Little Birdie. 
- Several non-product items (e.g. postage, gift vouchers and Amazon fees) appeared in the top revenue rankings, and were excluded from product level analysis.
- A subset of products were identified as low-volume, high price items. These are likely niche or luxury items, which may require pricing or marketing review. 

#### Geographic Trends
- The UK dominates total revenue, accounting for over £9M in sales. 
- After excluding the UK, top international markets include Netherlands, Eire and Germany; with significantly lower sales volumes. 

#### Time-Based Patterns
- Sales are heavily concentrated on Weekdays, especially Tuesdays through Thursdays.
- The most active shopping window is between 10am and 1pm, with minimal activity during evenings and weekends. 
- These patterns suggest that the customer base is largely engaged during business hours, which could inform promotional timings. 